In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests

In [2]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

In [3]:
# replacing with Tavily (google search) as 1000 free api monthly
from langchain_tavily import TavilySearch

search_tool = TavilySearch()

In [5]:
# lagnchain tool also available, but implementing custom tool

import os
OPENWEATHERMAP_API_KEY = os.environ['OPENWEATHERMAP_API_KEY']
@tool
def get_weather_data(city: str) -> str:
    """
    This function fetches the current weather data for a given city
    """
    url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHERMAP_API_KEY}'

    response = requests.get(url)

    return response.json()

In [6]:
llm = ChatOpenAI()

In [7]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [8]:
# Step 2: Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt
# it have tools list, usr query and thoughts


In [9]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [10]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)

In [11]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of Madhya Pradesh, then find it's current weather condition"})
print(response)     # Thought-Action-Observation loop

# starting loop
# agent sends -> actions and action input -> agent executor -> sends observation (output) and prev msgs to agent
# ending loop
# ends completely until agent says Final Answer



> Entering new AgentExecutor chain...
I should first search for the capital of Madhya Pradesh and then use the weather data function to get the current weather condition of that city.
Action: tavily_search
Action Input: Capital of Madhya Pradesh{'query': 'Capital of Madhya Pradesh', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://en.wikipedia.org/wiki/Madhya_Pradesh', 'title': 'Madhya Pradesh - Wikipedia', 'content': 'Its capital is Bhopal and the largest city is Indore. Other major cities includes Jabalpur, Gwalior, Ujjain and Sagar. Madhya Pradesh is the second largest', 'score': 0.98569, 'raw_content': None}, {'url': 'https://en.wikipedia.org/wiki/Bhopal', 'title': 'Bhopal - Wikipedia', 'content': "# Bhopal Bhopal Bhopal bhopal.city Flourishing around 1707, the city was the capital of the former Bhopal State, a princely state of the British ruled by the Nawabs of Bhopal until India's independence in 1947. Bhopal was also rated as the cleanes

In [12]:
response['output']

'The capital of Madhya Pradesh is Bhopal and the current weather condition in Bhopal is overcast clouds with a temperature of 298.44 Kelvin.'